# 通过随机扭动生成新的图片

<span style="color:red">import library<span>

In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [41]:
def randomMoveListGen(length, maxthresh):
    done = False
    while not done:
        tmp = np.random.choice([-1, 0, 1], [length])
        a = 0
        ret = []
        for i in tmp:
            a += i
            ret.append(a)
        ret = np.array(ret)
        ret[ret>maxthresh] = maxthresh
        ret[ret<-maxthresh] = -maxthresh
        cnt = len(ret[ret==maxthresh]) + len(ret[ret==-maxthresh])
        print cnt
        if cnt < length/3:
            done = True
    return ret

In [57]:
path = os.getcwd() + '/test3'